In [1]:
%matplotlib inline

In [167]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import spacy
from torch.utils.data import Dataset, DataLoader

In [482]:
# from spanish english dataset http://www.manythings.org/anki/spa-eng.zip
df = pd.read_csv("spa-eng/spa.txt", sep="\t", header=None)
df.columns = ["en", "es"]

In [483]:
df.head()

,en,es
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [484]:
df = df[df.en.str.count(" ").between(5, 7)]
df['en_tokens'] = df.en.str.split()
df['es_tokens'] = df.es.str.split()
df.shape

(47742, 4)

In [485]:
from collections import Counter, defaultdict
en_tokens = [w for w,n in
    Counter([token for index, row in df.en_tokens.items() for token in row]).most_common(5000)]
es_tokens = [w for w,n in
    Counter([token for index, row in df.es_tokens.items() for token in row]).most_common(5000)]

en_tokens.insert(0, '_bos_')
en_tokens.insert(1, '_pad_')
en_tokens.insert(2, '_eos_')
en_tokens.insert(3, '_unk')

es_tokens.insert(0, '_bos_')
es_tokens.insert(1, '_pad_')
es_tokens.insert(2, '_eos_')
es_tokens.insert(3, '_unk')

In [486]:
en_stoi = defaultdict(lambda: 3, {token:i for i,token in enumerate(en_tokens)})
en_itos = en_tokens
es_stoi = defaultdict(lambda: 3, {token:i for i,token in enumerate(es_tokens)})
es_itos = es_tokens

In [487]:
def add_padding(tokens, max_len):
    return tokens + ["_pad_"]*(max_len - len(tokens))
    
def add_padding_to(rows):
    max_len = rows.apply(lambda r: len(r)).max()
    return rows.apply(lambda row: add_padding(row, max_len))

df['es_tokens'] = add_padding_to(df.es_tokens)
df['en_tokens'] = add_padding_to(df.en_tokens)

en_phrases = [[en_stoi[token] for token in row] for index, row in df.en_tokens.items()]
es_phrases = [[es_stoi[token] for token in row] for index, row in df.es_tokens.items()]

In [488]:
e = nn.Embedding(len(en_tokens), 10, padding_idx=1)

In [489]:
class Seq2SeqDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return (
        np.array(self.x[idx]),
        np.array(self.y[idx])
    )
    def __len__(self): return len(self.x)

In [490]:
ds = Seq2SeqDataset(en_phrases, es_phrases)

In [491]:
dl = DataLoader(ds, batch_size=125, shuffle=True)

In [493]:
x, y = ds[0]
print("*", ' '.join([en_itos[o] for o in x if o != 1]))
print("*", ' '.join([es_itos[o] for o in y if o != 1]))

* I did it on my own.
* Lo hice yo sola.


In [496]:
class Seq2SeqModel(nn.Module):
    def __init__(self, enc_emb_size, enc_emb_dim, dec_emb_size, dec_emb_dim,
                enc_input_size, dec_input_size):
        super().__init__()
        
        self.dec_input_size = dec_input_size
        self.emb_enc = nn.Embedding(enc_emb_size, enc_emb_dim)
        self.gru_enc = nn.GRU(enc_emb_dim, 128, num_layers=2)
        self.out_enc = nn.Linear(128, dec_emb_dim, bias=False)
        
        self.emb_dec = nn.Embedding(dec_emb_size, dec_emb_dim)
        self.gru_dec = nn.GRU(dec_emb_dim, dec_emb_dim, num_layers=2)
        self.out_dec = nn.Linear(dec_emb_dim, dec_emb_size)
        
    def forward(self, input):
        input = input.permute(1, 0)
        sq, bs = input.size()
        o = self.emb_enc(input)
        o, h = self.gru_enc(o)
        h = self.out_enc(h)
        
        dec_in = torch.autograd.Variable(torch.zeros(bs).long())
        res = []
        for i in range(self.dec_input_size):
            o = self.emb_dec(dec_in).unsqueeze(0)
            o, h = self.gru_dec(o, h)
            o = self.out_dec(o[0])
            res.append(o)
            dec_in = o.data.max(1)[1]
            #if (dec_in == 1).all():
            #    break
        # import pdb; pdb.set_trace()
        return torch.stack(res)

model = Seq2SeqModel(len(en_itos), 300, len(es_itos), 300, 16, 17)

optim = torch.optim.RMSprop(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

for i,batch in enumerate(dl):
    x, y = batch
    print(x.size(), y.size())
    optim.zero_grad()
    o = model(x)
    print(o.size())
    loss = criterion(o.view(-1, 5004), y.permute(1, 0).contiguous().view(-1))
    # if i % 100 == 0:
    #    print(i, loss.item())
    loss.backward()
    optim.step()

torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.

torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.Size([17, 125, 5004])
torch.Size([125, 8]) torch.Size([125, 17])
torch.

KeyboardInterrupt: 

In [497]:
x,y = next(iter(dl))
probs = model(x)
preds = probs.max(2)[1].permute(1, 0)

for i in range(0,100):
    print("*", ' '.join([en_itos[o] for o in x.permute(1, 0)[:,i] if o != 1]))
    print("*", ' '.join([es_itos[o] for o in y.permute(1, 0)[:,i] if o != 1]))
    print("*", ' '.join([es_itos[o] for o in preds[:,i] if o!=1]))
    print()
preds.size()

* May I eat a little of it?
* ¿Puedo comer un poco?
* ¿Podría Ella Deberías Ellos ¿Quién Ella No ¿Cuándo No No Es No No Tom ¿Te El He Ellos Tom He He _unk Mi Mi Él Tú _unk No Él ¿Cuál Ellos Hablé No Estoy He No ¿Te No Yo No Ella Te He Ella Tengo _unk Ellos Tom No No Nuestro ¿Quieres No He Tom No ¿Dónde ¿Te Ellos Hablé El Hablé Tom No _unk Él Es ¿Te ¿Sabes ¿Dónde No Es ¿Cuándo Él ¿Sabes _unk _unk Hablé Hablé No No Tom Tom No Te Tom Tom El Mira Él No No Ella ¿Necesitas El Preferiría No No El Ella No Hablé Tom El Mary Esto No ¿Te _unk El ¿Hay Él No ¿Te No Él No Tom Siempre Él Tom Tengo Él El No

* She returned the book to the library.
* Ella regresó el libro a la biblioteca.
* traigo estaba _unk _unk quedan estaba que _unk que dejes pocas _unk que _unk _unk _unk _unk _unk no _unk quedan pocas abuelo no _unk que que no _unk _unk que que estoy un comprar _unk mandar nadie comprar _unk _unk un _unk que _unk _unk no pocas mandar destino mandar dejes _unk un mandar _unk quedan hay _unk _unk de

IndexError: index 17 is out of bounds for dimension 1 with size 17